In [1]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
sb.set() # set the default Seaborn style for graphics

In [2]:
chessdata=pd.read_csv('CleanedChessdata.csv')

C:\Users\rinoa\AppData\Local\Temp\ipykernel_17476\561380963.py:1: DtypeWarning: Columns (13,14,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,237,238,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614) have mixed types. Specify dtype option on import or set low_memo

In [4]:
chessdata.loc[chessdata.Result == "0-1", 'Result'] = "-1" #black win
chessdata.loc[chessdata.Result == "1-0", 'Result'] = "1" #white win
chessdata.loc[chessdata.Result == "1/2-1/2", 'Result'] = "0" #draw
# Import the encoder from sklearn
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
cat_predictors = chessdata[["Opening_new","Event"]]
# calculate average rating and rating difference as numeric predictors
chessdata['ELO Diff'] = chessdata['BlackElo'] - chessdata['BlackElo']
chessdata['Ave ELO']=.5*(chessdata['BlackElo'] +chessdata['BlackElo'])


In [5]:
ohe.fit(cat_predictors)
predictors_cat_ohe = pd.DataFrame(ohe.transform(cat_predictors).toarray(), 
                                  columns=ohe.get_feature_names_out(cat_predictors.columns))

Generate classification tree

In [6]:
# Import essential models and functions from sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.tree import plot_tree

In [8]:
chessdata.info()
chessData_num = chessdata[['ELO Diff','Ave ELO']]
chessData_res = chessdata['Result']
chessData_ohe = pd.concat([chessData_num, predictors_cat_ohe, chessData_res], 
                           sort = False, axis = 1).reindex(index=chessData_num.index)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197768 entries, 0 to 197767
Columns: 620 entries, Unnamed: 0 to Ave ELO
dtypes: float64(5), int64(6), object(609)
memory usage: 935.5+ MB


MemoryError: Unable to allocate 8.35 GiB for an array with shape (5665, 197768) and data type float64

In [18]:
s = chessdata.drop(['Opening_new', 'Opening'], axis = 1)

# Extract Response and Predictors
y = pd.DataFrame(s['Result'])
X = pd.DataFrame(s.drop('Result', axis = 1))

# Split the Dataset into Train and Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

# Decision Tree using Train Data
dectree = DecisionTreeClassifier(max_depth = 4)  # change max_depth to experiment
dectree.fit(X_train, y_train)                    # train the decision tree model

# Plot the trained Decision Tree
f = plt.figure(figsize=(24,24))
plot_tree(dectree, filled=True, rounded=True, 
          feature_names=X_train.column, 
          class_names=["Black","White"])

ValueError: could not convert string to float: '2019.05.02'